In [51]:
import pandas as pd
import numpy as np
import json
import sqlite3
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [104]:
orders_df = pd.read_csv('orders.csv')
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'], format='%d-%m-%Y')

print("Orders data shape:", orders_df.shape)
print("\nFirst 5 rows:")
print(orders_df.head())
print("\nColumns:", orders_df.columns.tolist())

Orders data shape: (10000, 6)

First 5 rows:
   order_id  user_id  restaurant_id order_date  total_amount  \
0         1     2508            450 2023-02-18        842.97   
1         2     2693            309 2023-01-18        546.68   
2         3     2084            107 2023-07-15        163.93   
3         4      319            224 2023-10-04       1155.97   
4         5     1064            293 2023-12-25       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  

Columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name']


In [53]:
with open('users.json', 'r') as f:
    users_data = json.load(f)
    users_df = pd.DataFrame(users_data)

print("Users Data Shape:", users_df.shape)
print("\nFirst 5 rows:")
print(users_df.head())
print("\nColumns:", users_df.columns.tolist())

Users Data Shape: (3000, 4)

First 5 rows:
   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold

Columns: ['user_id', 'name', 'city', 'membership']


In [54]:
restaurant_data = {
    'restaurant_id': [458, 309, 187, 224, 293, 501, 502, 503],
    'restaurant_name': [
        'New Foods Chinese',
        'Ruchi Curry House Multicuisine',
        'Spice Kitchen Punjabi',
        'Darbar Kitchen Non-Veg',
        'Royal Eatery South Indian',
        'Grand Cafe Punjabi',
        'Grand Restaurant South Indian',
        'Ruchi Mess Multicuisine'
    ],
    'cuisine': ['Chinese', 'Multicuisine', 'Punjabi', 'Indian', 'South Indian',
                'Punjabi', 'South Indian', 'Multicuisine'],
    'rating': [4.2, 4.5, 4.0, 4.7, 4.3, 3.8, 4.1, 3.5],
    'city': ['Chennai', 'Bangalore', 'Hyderabad', 'Mumbai', 'Chennai',
             'Pune', 'Hyderabad', 'Bangalore']
}

restaurant_df = pd.DataFrame(restaurant_data)

print("Restaurant Data Shape:", restaurant_df.shape)
print("\nFirst 5 rows:")
print(restaurant_df.head())

Restaurant Data Shape: (8, 5)

First 5 rows:
   restaurant_id                 restaurant_name       cuisine  rating  \
0            458               New Foods Chinese       Chinese     4.2   
1            309  Ruchi Curry House Multicuisine  Multicuisine     4.5   
2            187           Spice Kitchen Punjabi       Punjabi     4.0   
3            224          Darbar Kitchen Non-Veg        Indian     4.7   
4            293       Royal Eatery South Indian  South Indian     4.3   

        city  
0    Chennai  
1  Bangalore  
2  Hyderabad  
3     Mumbai  
4    Chennai  


In [108]:
merged_df = pd.merge(orders_df, users_df, on='user_id', how='left')
merged_df = pd.merge(merged_df, restaurant_df, on='restaurant_id', how='left')
merged_df = merged_df.rename(columns={
    'city_x': 'user_city',
    'city_y': 'restaurant_city',
    'restaurant_name_x': 'order_restaurant_name',
    'restaurant_name_y': 'restaurant_name'
})

print("Merged Data Shape:", merged_df.shape)
print("\nFirst 5 rows:")
print(merged_df.head())
print("\nColumns:", merged_df.columns.tolist())

Merged Data Shape: (10000, 13)

First 5 rows:
   order_id  user_id  restaurant_id order_date  total_amount  \
0         1     2508            450 2023-02-18        842.97   
1         2     2693            309 2023-01-18        546.68   
2         3     2084            107 2023-07-15        163.93   
3         4      319            224 2023-10-04       1155.97   
4         5     1064            293 2023-12-25       1321.91   

            order_restaurant_name       name  user_city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

                  restaurant_name       cuisine  rating restaurant_city  
0                             NaN           NaN     NaN             NaN  


City with highest revenue from Gold members

In [110]:
gold_members_df = merged_df[merged_df['membership'] == 'Gold']
city_revenue = gold_members_df.groupby('user_city')['total_amount'].sum().reset_index()
highest_city = city_revenue.loc[city_revenue['total_amount'].idxmax()]

print(f"Answer 1: {highest_city['user_city']} has the highest revenue from Gold members")

Answer 1: Chennai has the highest revenue from Gold members


2 - Cuisine with highest average order value

In [111]:
cuisine_df = merged_df.dropna(subset=['cuisine'])
cuisine_avg_value = cuisine_df.groupby('cuisine')['total_amount'].mean().reset_index()
highest_cuisine = cuisine_avg_value.loc[cuisine_avg_value['total_amount'].idxmax()]

print(f"Answer 2: {highest_cuisine['cuisine']} has the highest average order value")

Answer 2: Indian has the highest average order value


3 - Users with total orders > ₹1000


In [107]:
user_total = merged_df.groupby('user_id')['total_amount'].sum().reset_index()
high_value_users = user_total[user_total['total_amount'] > 1000]
count_high_value = len(high_value_users)

if count_high_value < 500:
    answer = "< 500"
elif 500 <= count_high_value <= 1000:
    answer = "500 – 1000"
elif 1001 <= count_high_value <= 2000:
    answer = "1000 – 2000"
else:
    answer = "> 2000"

print(f"Answer 3: {answer}")

Answer 3: > 2000


 4 - Rating range with highest revenue

In [112]:
merged_df["rating_range"] = pd.cut(
    merged_df["rating"],
    bins=[3,3.5,4,4.5,5],
    labels=["3.0-3.5","3.6-4.0","4.1-4.5","4.6-5.0"]
)

rating_revenue = merged_df.groupby("rating_range")["total_amount"].sum().reset_index()
best_range = rating_revenue.loc[rating_revenue["total_amount"].idxmax()]
print(f"Answer 4: {best_range['rating_range']}")


Answer 4: 4.1-4.5


5 - City with highest average order value for Gold members

In [114]:
gold_df = merged_df[merged_df['membership'] == 'Gold']
city_avg_value = gold_df.groupby('user_city')['total_amount'].mean().reset_index()
highest_city_avg = city_avg_value.loc[city_avg_value['total_amount'].idxmax()]

print(f"Answer 5: {highest_city_avg['user_city']} has the highest average order value for Gold members")

Answer 5: Chennai has the highest average order value for Gold members


6 - Cuisine with fewest restaurants but significant revenue

In [81]:
complete_df = merged_df.dropna(subset=['restaurant_name', 'cuisine'])
restaurant_count = complete_df.groupby('cuisine')['restaurant_id'].nunique().reset_index()
restaurant_count.columns = ['cuisine', 'restaurant_count']
cuisine_revenue = complete_df.groupby('cuisine')['total_amount'].sum().reset_index()
cuisine_analysis = pd.merge(restaurant_count, cuisine_revenue, on='cuisine')
cuisine_analysis['revenue_per_restaurant'] = cuisine_analysis['total_amount'] / cuisine_analysis['restaurant_count']
cuisine_analysis = cuisine_analysis.sort_values(['restaurant_count', 'revenue_per_restaurant'],
                                                ascending=[True, False])
print("Cuisine Analysis (fewest restaurants with significant revenue):")
print(cuisine_analysis)
lowest_cuisine = cuisine_analysis.iloc[0]['cuisine']
print(f"\nAnswer 6: {lowest_cuisine} has the lowest number of distinct restaurants but significant revenue")

Cuisine Analysis (fewest restaurants with significant revenue):
        cuisine  restaurant_count  total_amount  revenue_per_restaurant
3       Punjabi                 1      18322.97                18322.97
1        Indian                 1      17523.73                17523.73
2  Multicuisine                 1      16518.82                16518.82
4  South Indian                 1      15824.75                15824.75
0       Chinese                 1      13060.59                13060.59

Answer 6: Punjabi has the lowest number of distinct restaurants but significant revenue


7 - Percentage of orders by Gold members

In [118]:
total_orders = len(merged_df)
gold_orders = len(merged_df[merged_df['membership'] == 'Gold'])
gold_percentage = (gold_orders / total_orders) * 100
rounded_percentage = round(gold_percentage)
print(f"Percentage: {gold_percentage:.2f}%")
print(f"Rounded: {rounded_percentage}%")

Percentage: 49.87%
Rounded: 50%


8 - Restaurant with highest average but <20 orders

In [82]:
rest_stats = merged_df.groupby("restaurant_name") \
.agg(avg=("total_amount","mean"),
     cnt=("order_id","count")) \
.reset_index()

filtered = rest_stats[rest_stats["cnt"] < 20]
best_rest = filtered.loc[filtered["avg"].idxmax()]

print(filtered.sort_values("avg",ascending=False).head())
print(f"\nAnswer 8: {best_rest['restaurant_name']}")


     restaurant_name         avg  cnt
1  New Foods Chinese  687.399474   19

Answer 8: New Foods Chinese


9 - Combination with highest revenue

In [83]:
combo = merged_df.groupby(["membership","cuisine"])["total_amount"].sum().reset_index()
best_combo = combo.loc[combo["total_amount"].idxmax()]

print(combo.sort_values("total_amount",ascending=False))
print(f"\nAnswer 9: {best_combo['membership']} + {best_combo['cuisine']}")


  membership       cuisine  total_amount
1       Gold        Indian      11209.90
8    Regular       Punjabi      10401.89
2       Gold  Multicuisine       9105.61
9    Regular  South Indian       7954.40
3       Gold       Punjabi       7921.08
4       Gold  South Indian       7870.35
7    Regular  Multicuisine       7413.21
5    Regular       Chinese       6616.01
0       Gold       Chinese       6444.58
6    Regular        Indian       6313.83

Answer 9: Gold + Indian


10 - Quarter with highest revenue

In [65]:
merged_df['quarter'] = merged_df['order_date'].dt.quarter
quarter_map = {1: 'Q1 (Jan–Mar)', 2: 'Q2 (Apr–Jun)',
               3: 'Q3 (Jul–Sep)', 4: 'Q4 (Oct–Dec)'}
merged_df['quarter_label'] = merged_df['quarter'].map(quarter_map)
quarter_revenue = merged_df.groupby('quarter_label')['total_amount'].sum().reset_index()
highest_quarter = quarter_revenue.loc[quarter_revenue['total_amount'].idxmax()]

print("Revenue by Quarter:")
print(quarter_revenue.sort_values('total_amount', ascending=False))
print(f"\nAnswer 10: {highest_quarter['quarter_label']}")

Revenue by Quarter:
  quarter_label  total_amount
2  Q3 (Jul–Sep)    2037385.10
3  Q4 (Oct–Dec)    2018263.66
0  Q1 (Jan–Mar)    2010626.64
1  Q2 (Apr–Jun)    1945348.72

Answer 10: Q3 (Jul–Sep)


1-How many total orders were placed by users with Gold membership?


In [88]:
gold_orders = merged_df[merged_df['membership'] == 'Gold']
total_gold_orders = len(gold_orders)

print(f"Total orders placed by Gold members: {total_gold_orders}")

Total orders placed by Gold members: 4987


2-What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

In [102]:
hyd_orders = merged_df[merged_df['user_city'] == 'Hyderabad']
total_hyd_revenue = hyd_orders['total_amount'].sum()
rounded_revenue = round(total_hyd_revenue)

print(f"Total revenue from Hyderabad: ₹{total_hyd_revenue:.2f}")
print(f"Rounded to nearest integer: ₹{rounded_revenue}")

Total revenue from Hyderabad: ₹1889366.58
Rounded to nearest integer: ₹1889367


3 - Distinct users who placed at least one order

In [91]:
distinct_users = merged_df["user_id"].nunique()
print(f"Distinct users who placed orders: {distinct_users}")

Distinct users who placed orders: 2883


4-What is the average order value (rounded to 2 decimals) for Gold members?

In [94]:
gold_avg = merged_df[merged_df["membership"]=="Gold"]["total_amount"].mean()
print(f"Average order value (Gold): {round(gold_avg,2)}")

Average order value (Gold): 797.15


5-How many orders were placed for restaurants with rating ≥ 4.5?

In [98]:
high_rating_orders = merged_df[merged_df['rating'] >= 4.5]
count_high_rating = len(high_rating_orders)

print(f"Number of orders with rating ≥ 4.5: {count_high_rating}")

Number of orders with rating ≥ 4.5: 40


6-How many orders were placed in the top revenue city among Gold members only?

In [101]:
gold_orders = merged_df[merged_df['membership'] == 'Gold']
city_revenue = gold_orders.groupby('user_city')['total_amount'].sum().reset_index()
top_city = city_revenue.loc[city_revenue['total_amount'].idxmax(), 'user_city']
orders_in_top_city = gold_orders[gold_orders['user_city'] == top_city]
count_top_city_orders = len(orders_in_top_city)

print(f"Number of orders placed in {top_city} by Gold members: {count_top_city_orders}")

Number of orders placed in Chennai by Gold members: 1337
